---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
# % matplotlib inline

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    Algorithm = [0,0,0,0,0]
    for G in P1_Graphs:
        degrees=G.degree()
        degree_values=sorted(set(degrees.values()))
        histogram=[list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]

    #     plt.plot(degree_values, histogram, 'o')
    #     plt.xlabel('Degree')
    #     plt.ylabel('Fraction of Nodes')
    #     plt.xscale('log')
    #     plt.yscale('log')
    #     plt.show()

    # P1_Graphs[0] and P1_Graphs[3] are PA
    Algorithm[0]='PA'
    Algorithm[3]='PA'

    for G in [P1_Graphs[1],P1_Graphs[2],P1_Graphs[4]]:
        if nx.average_clustering(G)>0.1:
            Algorithm[P1_Graphs.index(G)]='SW_L'
        else:
            Algorithm[P1_Graphs.index(G)]='SW_H'
    return Algorithm# Your Answer Here

graph_identification()


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    # Your Code Here
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV

    df = pd.DataFrame(index=G.nodes())
    df['Department']=pd.Series(nx.get_node_attributes(G,'Department'))
    df['ManagementSalary']=pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    df['clustering']=pd.Series(nx.clustering(G))
    df['degree']=pd.Series(G.degree())
    df['degree_centrality']=pd.Series(nx.degree_centrality(G))
    df['closeness_centrality']=pd.Series(nx.closeness_centrality(G,
                                          normalized=True))
    df['betweenness_centrality']=pd.Series(nx.betweenness_centrality(G,
                                          normalized=True))
    df['PR'] = pd.Series(nx.pagerank(G))
    
    # Prediction
    df_train_test=df[~pd.isnull(df['ManagementSalary'])]
    df_y_train_test=df_train_test['ManagementSalary']
    df_X_train_test=df_train_test.loc[:,df_train_test.columns!='ManagementSalary']
    scaler=MinMaxScaler()
    df_X_train_test_scaled=scaler.fit_transform(df_X_train_test)
    X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(df_X_train_test_scaled,df_y_train_test,random_state=0)

    df_target=df[pd.isnull(df['ManagementSalary'])]
    df_y_target=df_target['ManagementSalary']
    df_X_target=df_target.loc[:,df_target.columns!='ManagementSalary']
    df_X_target_scaled=scaler.transform(df_X_target)

    clf=RandomForestClassifier(max_depth=10)
    grid_values={'n_estimators':[5,10,15,20]}
    grid_clf_custom=GridSearchCV(clf,param_grid=grid_values,scoring='roc_auc')
    grid_clf_custom.fit(X_train_scaled,y_train)
    #grid_clf_custom.best_score_
    
    return pd.Series(grid_clf_custom.predict_proba(df_X_target_scaled)[:,1],index=df_target.index)# Your Answer Here

salary_predictions()

1       0.002083
2       1.000000
5       1.000000
8       0.368988
14      0.000000
18      0.150000
27      0.000000
30      0.505000
31      0.563542
34      0.050000
37      0.100000
40      0.102083
45      0.002083
54      0.398750
55      0.600000
60      0.299792
62      1.000000
65      0.950000
77      0.100000
79      0.044444
97      0.000000
101     0.000000
103     0.600595
108     0.050000
113     0.050000
122     0.000000
141     0.300000
142     1.000000
144     0.000000
145     0.301613
          ...   
913     0.000000
914     0.000000
915     0.000000
918     0.000000
923     0.000000
926     0.000000
931     0.050000
934     0.000000
939     0.000000
944     0.000000
945     0.000000
947     0.100000
950     0.121875
951     0.000000
953     0.000000
959     0.000000
962     0.000000
963     0.121875
968     0.000000
969     0.000000
974     0.000000
984     0.000000
987     0.000000
989     0.000000
991     0.000000
992     0.000000
994     0.000000
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
def new_connections_predictions():
    
    # Your Code Here
    # Your Code Here
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV

    df = future_connections

    df['Common Neighbors'] = df.index.map(lambda tpl: len(list(nx.common_neighbors(G, tpl[0], tpl[1]))))
    df['Jaccard Coefficient'] = [i[2] for i in list(nx.jaccard_coefficient(G, df.index))]
    df['Resource Allocation']= [i[2] for i in list(nx.resource_allocation_index(G, df.index))]
    df['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    df['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    for node in G.nodes():
        G.node[node]['community']=G.node[node]['Department']
    df['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, df.index)]
    df['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, df.index)]


    # Prediction
    df_train_test=df[~pd.isnull(df['Future Connection'])]
    df_y_train_test=df_train_test['Future Connection']
    df_X_train_test=df_train_test.loc[:,df_train_test.columns!='Future Connection']
    scaler=MinMaxScaler()
    df_X_train_test_scaled=scaler.fit_transform(df_X_train_test)
    X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(df_X_train_test_scaled,df_y_train_test,random_state=0)

    df_target=df[pd.isnull(df['Future Connection'])]
    df_y_target=df_target['Future Connection']
    df_X_target=df_target.loc[:,df_target.columns!='Future Connection']
    df_X_target_scaled=scaler.transform(df_X_target)

    clf=RandomForestClassifier(max_depth=10)
    grid_values={'n_estimators':[5,10,15,20]}
    grid_clf_custom=GridSearchCV(clf,param_grid=grid_values,scoring='roc_auc')
    grid_clf_custom.fit(X_train_scaled,y_train)
    #grid_clf_custom.best_score_
    
    return pd.Series(grid_clf_custom.predict_proba(df_X_target_scaled)[:,1],index=df_target.index)# Your Answer Here

new_connections_predictions()

(107, 348)    0.023884
(542, 751)    0.012693
(20, 426)     0.523662
(50, 989)     0.012693
(942, 986)    0.013782
(324, 857)    0.012693
(13, 710)     0.163798
(19, 271)     0.157847
(319, 878)    0.012599
(659, 707)    0.012693
(49, 843)     0.012599
(208, 893)    0.012693
(377, 469)    0.011579
(405, 999)    0.014730
(129, 740)    0.011454
(292, 618)    0.017061
(239, 689)    0.012599
(359, 373)    0.014790
(53, 523)     0.472123
(276, 984)    0.012599
(202, 997)    0.012257
(604, 619)    0.032600
(270, 911)    0.012599
(261, 481)    0.063676
(200, 450)    0.993555
(213, 634)    0.012693
(644, 735)    0.026184
(346, 553)    0.013183
(521, 738)    0.011990
(422, 953)    0.014504
                ...   
(672, 848)    0.012599
(28, 127)     0.998348
(202, 661)    0.012785
(54, 195)     1.000000
(295, 864)    0.012693
(814, 936)    0.012693
(839, 874)    0.013782
(139, 843)    0.012693
(461, 544)    0.013033
(68, 487)     0.011563
(622, 932)    0.012693
(504, 936)    0.017153
(479, 528) 